In [ ]:
import sys
import os
import cv2
from PyQt5 import QtCore
from PyQt5 import QtGui
from PyQt5.QtCore import pyqtSlot,QUrl
from PyQt5.QtGui import QImage ,QPixmap
from PyQt5.uic import loadUi
from PyQt5 import QtWidgets, uic
from PyQt5.QtWidgets import *
from PyQt5.QtMultimedia import *
import import_ipynb
from CNN_Model import CNN_Model
from Train_Test_Data import train_test_data


class MBRS(QtWidgets.QMainWindow):
    def __init__(self):
        super(MBRS, self).__init__()
        loadUi('MBRSui.ui',self)
        self.logic=0
        self.filename="test images/default.jpg"
        
        self.cnn_model_obj=CNN_Model()
        self.train_test_data_obj = train_test_data()
        
        self.VolumeSlider.setRange(0, 100) 
        self.VolumeSlider.setValue(40)
        self.mediaplayer =QMediaPlayer() 
        self.mediaplayer.setVolume(40)
        self.i=1
        self.predictionResult="none"
        self.Qmediaplayerplaylist = QMediaPlaylist(self.mediaplayer)
        
        self.OpencameraButton.clicked.connect(self.openCamera)
        self.OpenFileButton.clicked.connect(self.openFile)
        self.CaptureImageButton.clicked.connect(self.captureImage)
        self.CreateplaylistButton.clicked.connect(self.playListFunction)
        self.mediaplayer.stateChanged.connect(self.mediaStateChanged)
        self.PlayButton.clicked.connect(self.play)
        self.PauseButton.clicked.connect(self.stop)
        self.PreviousButton.clicked.connect(self.previousMusic)
        self.NextButton.clicked.connect(self.nextMusic)
        self.mediaplayer.positionChanged.connect(self.position_changed)
        self.mediaplayer.durationChanged.connect(self.duration_changed)
        self.MusicSlider.sliderMoved.connect(self.set_position)
        self.VolumeSlider.sliderMoved.connect(self.set_volume)
        self.UpdateDatasetButton.triggered.connect(self.train_test_data_obj.update_Train_Test_Data)
        self.UpdateModelButton.triggered.connect(self.cnn_model_obj.update_model)
        self.listWidget.currentRowChanged.connect(self.mediaChange)
        

        
    def openCamera(self):
        self.logic=1
        cap=cv2.VideoCapture(0)
        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

        while (cap.isOpened()):
            ret, frame=cap.read()
                            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=3,minSize=(48, 48))
            if ret== True:
                
                for (x, y, w, h) in faces:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)   
                       
                self.displayImage(frame,1)
                cv2.waitKey()    

                if(self.logic==2):
                    self.logic=1
                    for (x, y, w, h) in faces:
                        roi_color = frame[y:y + h, x:x + w]
                        print("[INFO] Object found. Saving locally.")  
                        status =cv2.imwrite('test images/faces_detected.jpg', roi_color)
                        print("[INFO] Image faces_detected.jpg written to filesystem: ", status)
                    
                    pixmap=QPixmap("test images/faces_detected.jpg")
                    self.PicBox.setPixmap(pixmap)
                    self.filename="test images/faces_detected.jpg"
                    cap.release()
                    cv2.destroyAllWindows() 
                    
                if(self.logic==3):
                    self.logic=1
                    cap.release()
                    cv2.destroyAllWindows()
            else:
                print('return not found')
        cap.release()
        cv2.destroyAllWindows()
       
    def displayImage(self,img,window=1):
        qformat = QImage.Format_Indexed8
        
        if len(img.shape) == 3:
            if(img.shape[2]) == 4:
                qformat =QImage.Format_RGBA888
                
            else:
                qformat = QImage.Format_RGB888
        img = QImage(img, img.shape[1], img.shape[0],qformat)
        img = img.rgbSwapped()
        self.PicBox.setPixmap(QPixmap.fromImage(img))
        self.PicBox.setAlignment(QtCore.Qt.AlignHCenter | QtCore.Qt.AlignVCenter)
        
    def captureImage(self):
        self.logic=2
        
    def openFile(self):
        self.logic=3
        options = QFileDialog.Options()
        options |= QFileDialog.DontUseNativeDialog
        self.filename, _ = QFileDialog.getOpenFileName(self,"Select Image", "","JPG (*.jpg);;PNG (*.png);;All Files (*)", options=options)
        if self.filename:
            print(self.filename) 
        pixmap=QPixmap(self.filename)
        self.PicBox.setPixmap(pixmap)

#-----------------create playlist and show it in playlistview---------------------------


    def playListFunction(self):
#------------Predict model on detected faces-----------------------------
    
        print("predict data on model....")
        self.predictionResult=self.cnn_model_obj.Model_prediction(self.filename)
        #status="Happy"
        print(self.predictionResult)
        self.list=[]
        self.Qmediaplayerplaylist.clear()
        self.listWidget.clear() 
        for root, dirs, files in os.walk('Musics/'+self.predictionResult):
            for filename in files: 
                self.list.append(filename)            
        
                self.Qmediaplayerplaylist.addMedia(QMediaContent(QUrl.fromLocalFile(root+"/"+filename))) 
              
        self.Qmediaplayerplaylist.setCurrentIndex(1)

        self.mediaplayer.setPlaylist(self.Qmediaplayerplaylist)
        print(self.list)
        self.listWidget.addItems(self.list)  
        self.listWidget.setCurrentRow(0)
        
            
#------------------play function to play music--------------------------------------------------        
    def play(self):
        
        if self.mediaplayer.state() == QMediaPlayer.PlayingState:
            self.mediaplayer.pause()
        else:
            self.mediaplayer.play()
            
#-------------------stop function to stop the musci-----------------------
    def stop(self):      
        self.mediaplayer.stop()
        

#--------------------to play next music------------------------------    
    def nextMusic(self): 
         
        if self.i < self.Qmediaplayerplaylist.mediaCount():           
            if self.mediaplayer.state() ==QMediaPlayer.PlayingState:        
                self.Qmediaplayerplaylist.next()
                ind=self.Qmediaplayerplaylist.currentIndex()
                self.listWidget.setCurrentRow(ind)
                self.i=self.i + 1
            print(self.i)
        else :
            self.i=1
            self.Qmediaplayerplaylist.setCurrentIndex(0)
            ind=self.Qmediaplayerplaylist.currentIndex()
            self.listWidget.setCurrentRow(ind)

                    
        row=self.listWidget.currentRow()         
        itm =self.listWidget.item(row)       
        print(itm.text())
        
#-------------------one music step back-----------------------------------   
    def previousMusic(self):      
        if self.i > 1:
            if self.mediaplayer.state() == QMediaPlayer.PlayingState:
                self.Qmediaplayerplaylist.previous()
                ind=self.Qmediaplayerplaylist.currentIndex()
                self.listWidget.setCurrentRow(ind)  
                self.i=self.i - 1

            print(self.i)
        else :
            self.i=self.Qmediaplayerplaylist.mediaCount()
            self.Qmediaplayerplaylist.setCurrentIndex(self.i-1)
            ind=self.Qmediaplayerplaylist.currentIndex()
            self.listWidget.setCurrentRow(ind)

                    
        row=self.listWidget.currentRow()         
        itm =self.listWidget.item(row)       
        print(itm.text())            
#---------------------change icon of play button when media state change-------
    def mediaStateChanged(self, state):
        if self.mediaplayer.state() == QMediaPlayer.PlayingState:
            self.PlayButton.setIcon(
                    self.style().standardIcon(QStyle.SP_MediaPause))
        else:
            self.PlayButton.setIcon(
                    self.style().standardIcon(QStyle.SP_MediaPlay))
 
           
    def mediaChange(self):
        
        if self.i > 0 & self.i < self.Qmediaplayerplaylist.mediaCount():
            print("media change")
            row=self.listWidget.currentRow()         
            itm =self.listWidget.item(row)       
            self.MusicDetailsLabel.setText("<html><head/><body><p>Expression :</p><p>"+self.predictionResult+"</p><p>Music : </p><p>"+itm.text()+"</p></body></html>")

            
#--------------to change the slider position change according to music---------------
    def position_changed(self,position):
        self.MusicSlider.setValue(position)
        
    def duration_changed(self,duration):
        self.MusicSlider.setRange(0, duration)  
    
    def set_position(self,position):
        self.mediaplayer.setPosition(position)
        
    def set_volume(self, volume):
        self.mediaplayer.setVolume(volume)
            
app = QtWidgets.QApplication(sys.argv)
window = MBRS()

window.show()   
app.exec_()

importing Jupyter notebook from CNN_Model.ipynb
importing Jupyter notebook from Train_Test_Data.ipynb
[INFO] Object found. Saving locally.
[INFO] Image faces_detected.jpg written to filesystem:  True
predict data on model....
test images/faces_detected.jpg
Predicted: class=3
happy
['barish.mp3', 'mercy.mp3', 'musafir.mp3', 'o sathi.mp3', 'phir bhi.mp3']
media change
[INFO] Object found. Saving locally.
[INFO] Image faces_detected.jpg written to filesystem:  True
predict data on model....
test images/faces_detected.jpg
Predicted: class=2
fear
['02. Rab Na Kare.mp3', 'musafir.mp3', 'o sathi.mp3', 'phir bhi.mp3']
media change
